In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as T
import torch.optim as optim
import torchvision.models as models
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vgg = models.vgg11()
print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [4]:
class myVGG(nn.Module):
    def __init__(self):
        super(myVGG,self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3,64,kernel_size= 3,padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2,2,0,1),

            nn.Conv2d(64,128,kernel_size= 3,padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.MaxPool2d(2,2,0,1),

            nn.Conv2d(128,256,kernel_size= 3,padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.Conv2d(256,256,kernel_size= 3,padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.MaxPool2d(2,2,0,1,),

            nn.Conv2d(256,512,kernel_size= 3,padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(512,512,kernel_size= 3,padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            #512x2x2
            nn.MaxPool2d(2,2,0,1),
            nn.Conv2d(512,512,kernel_size= 3,padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.Conv2d(512,512,kernel_size = 3,padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.MaxPool2d(2,2,0,1),
        )

        # Initialize weights
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                n = m.kernel_size[0]*m.kernel_size[1]*m.out_channels
                m.weight.data.normal_(0,math.sqrt(2./n))
                m.bias.data.zero_()
                
        # self.avgpool = nn.AdaptiveAvgPool2d(output_size=(2,2))
        self.classifier = nn.Sequential(
            # nn.Linear(in_features=2*2*512,out_features=256,bias=True),
            # nn.ReLU(inplace=True),
            # nn.Dropout(p=0.5,inplace=False),    
            # nn.Linear(in_features=256,out_features=256,bias=True),
            # nn.ReLU(True),
            # nn.Dropout(p=0.5,inplace=False),
            # nn.Linear(in_features=256,out_features=10,bias=True)
            nn.Dropout(),
            nn.Linear(512,512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512,512),
            nn.ReLU(True),
            nn.Linear(512,10),
        )

    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)
        return x

In [5]:
transforms = T.Compose([
                        T.RandomCrop((32,32),padding=(4,4)),
                        T.RandomHorizontalFlip(p=0.5),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                        
])

trainset = dsets.CIFAR10(root = '/content/drive/MyDrive/lab/VGGNET/datasets',train=True,transform=transforms,download=True)
testset = dsets.CIFAR10(root='/content/drive/MyDrive/lab/VGGNET/datasets',train=False,transform=T.Compose([         #T.RandomCrop((32,32),padding=(4,4)),
                                                                                                                    # T.RandomHorizontalFlip(p=0.5),
                                                                                                                    T.ToTensor(),
                                                                                                                    T.Normalize(mean=[0.485,0.456,0.406],
                                                                                                                                std = [0.229,0.224,0.225])
]),download=True)
classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

Files already downloaded and verified
Files already downloaded and verified


In [6]:
trainLoader = torch.utils.data.DataLoader(trainset,batch_size = 128,shuffle = True)
testLoader = torch.utils.data.DataLoader(testset,batch_size =128,shuffle=False,drop_last=True)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = myVGG()
net = net.to(device)

In [8]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(net.parameters(),lr=0.05,momentum=0.9,weight_decay=5e-4)
# scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

In [9]:

def check_accuracy(network,testLoader):
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testLoader:
            images,labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = network(images)
            #
            _,predicted = torch.max(outputs,1)
            c = (predicted == labels).squeeze()
            for i in range(128):
                label = labels[i]
                class_correct[label] +=c[i].item()
                class_total[label] +=1
    accuracy_sum = 0
    for i in range(10):
        temp = 100* class_correct[i] / class_total[i]
        accuracy_sum += temp
    return accuracy_sum / 10

In [10]:

def show_accuracy(network,testLoader):
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testLoader:
            images,labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = network(images)
            #
            _,predicted = torch.max(outputs,1)
            c = (predicted == labels).squeeze()
            for i in range(128):
                label = labels[i]
                class_correct[label] +=c[i].item()
                class_total[label] +=1
    accuracy_sum = 0
    for i in range(10):
        temp = 100* class_correct[i] / class_total[i]
        print('Accuracy of %5s : %2d %%'%(classes[i],temp))
        accuracy_sum += temp
    print('Accuracy average: ',accuracy_sum / 10)

In [11]:
def adjust_learning_rate(optimizer,epoch,lr):
    lr = lr*(0.5 **(epoch //30))
    for param_group in optimizer.param_groups:
        param_group['lr']=lr
    return lr

In [12]:
epochs_accuracy_list = [1e9]
epochs_accuracy = 0
lr = 0.05
for epoch in range(200):
    running_loss = 0.0
    avg_loss=0.0
    for i,data in enumerate(trainLoader,0):
        #get the inputs
        inputs,labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        #zero the parameter gradients
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # # print statistics
        # running_loss += loss.item()
        # avg_loss += loss.item()
        # if i%50 ==49:
        #     print('[%d,%5d] loss: %.3f'%(epoch+1,i+1,running_loss/50))
        #     running_loss=0.0

    # if abs(pre_avg_loss-avg_loss)<=0.01 and epoch!=0:
    #     print("learning rate 50% reduced.")
    #     scheduler.step()
    acc = check_accuracy(net,testLoader)
    epochs_accuracy += acc
    print("epoch: {} | Accuracy: {}".format(epoch,acc))
    if epoch % 5 == 0:
        show_accuracy(net,testLoader)
        epochs_accuracy_list.append(epochs_accuracy/5)
        epochs_accuracy = 0

print('Finished Training')



epoch: 0 | Accuracy: 49.462444965372285
Accuracy of airplane : 35 %
Accuracy of automobile : 46 %
Accuracy of  bird : 30 %
Accuracy of   cat : 14 %
Accuracy of  deer : 41 %
Accuracy of   dog : 44 %
Accuracy of  frog : 73 %
Accuracy of horse : 63 %
Accuracy of  ship : 68 %
Accuracy of truck : 74 %
Accuracy average:  49.26225437367748
epoch: 1 | Accuracy: 62.09555878636489
epoch: 2 | Accuracy: 65.98003807973978
epoch: 3 | Accuracy: 71.46760918863642
epoch: 4 | Accuracy: 73.9502469034261
epoch: 5 | Accuracy: 75.81359701144311
Accuracy of airplane : 78 %
Accuracy of automobile : 81 %
Accuracy of  bird : 59 %
Accuracy of   cat : 43 %
Accuracy of  deer : 83 %
Accuracy of   dog : 71 %
Accuracy of  frog : 82 %
Accuracy of horse : 80 %
Accuracy of  ship : 83 %
Accuracy of truck : 92 %
Accuracy average:  75.77343632923642
epoch: 6 | Accuracy: 77.97984170090578
epoch: 7 | Accuracy: 78.42167768892907
epoch: 8 | Accuracy: 79.6636433655783
epoch: 9 | Accuracy: 79.93492488788307
epoch: 10 | Accuracy:

KeyboardInterrupt: ignored

In [ ]:
# with torch.no_grad():
#     for data in testLoader:
#         images,labels = data
#         images = images.cuda()
#         labels = labels.cuda()
#         outputs = net(images)
#         print(outputs.shape)
#         #values , indices -> max 값, max값이 나온 index => 따라서 indices 가 predicted label 이다.
#         _,predicted = torch.max(outputs,1)
#         # predicted == labels 에 대한 T/F인 Tensor return.
#         c = (predicted == labels).squeeze()
#         print(c)
#         for i in range(16):
#             # i번째 image 에 대한 label 값
#             label = labels[i]
#             # 해당 label에서 c가 True면 1이 + 되고, False면 0이 +된다.
#             class_correct[label] += c[i].item()
#             # 해당 label의 total 개수에 +1 을 해준다.
#             class_total[label] += 1
# accuracy_sum =0
# for i in range(10):
#     temp = 100 * class_correct[i] / class_total[i]
#     print('Accuracy of %5s : %2d %%'%(classes[i],temp))
#     accuracy_sum += temp
# print('Acuuray average: ',accuracy_sum/10)

In [ ]:
print(epochs_accuracy_list[1:])

[10.040672865322454, 70.15013582605908, 79.31101481981395, 81.79505396785984, 82.59884132969259, 83.4814135250621, 83.15882949889385, 83.80298047842317, 83.50835712860939, 84.1718935578453, 84.45785644493183, 84.59597714384635, 84.7158831296548, 84.63045622414111, 84.66939719600728, 84.69449581442356, 84.39125527821275, 84.36223007114548, 84.70470497465342, 84.91304034527408, 84.89556122273231]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data = [10.040672865322454, 70.15013582605908, 79.31101481981395, 81.79505396785984, 82.59884132969259, 83.4814135250621, 83.15882949889385, 83.80298047842317, 83.50835712860939, 84.1718935578453, 84.45785644493183, 84.59597714384635, 84.7158831296548, 84.63045622414111, 84.66939719600728, 84.69449581442356, 84.39125527821275, 84.36223007114548, 84.70470497465342, 84.91304034527408, 84.89556122273231]

In [ ]:
x=[]
for i in range(len(data)):
    x.append(i*5)

In [ ]:
plt.title("VGG11 + CIFAR10")
plt.plot(x,data)
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.show()